In [1]:
using StatsBase
using Graphs, MetaGraphs
using Makie
using BenchmarkTools

In [2]:
# utils
Base.getindex(g::MetaDiGraph, i::Integer) = g.vprops[i]
Base.getindex(g::MetaDiGraph, i::Integer, j::Integer) = g.eprops[Edge(i,j)]

function color_print(io::IO, chars, colors)
    for (c, col) in zip(chars, colors)
        printstyled(io, string(c); color=col)
    end
end

function entropy(pvec::Vector{Float64})
    return mapreduce(p -> (p > 0) ? -p * log(p) : 0.0, +, pvec)
end

entropy (generic function with 1 method)

In [3]:
const ITYPE = UInt16 # depends on the number of words
const NLETTERS = 5

@enum TileColor begin # as per WORDLE
    GREEN = 0
    YELLOW = 1
    GREY = 2
end
const NCOLORS = length(instances(TileColor))

get_color(tc::TileColor) = (tc == GREEN) ? :green : (tc == YELLOW) ? :yellow : :grey

# now, the overall feedback
const Feedback = NTuple{NLETTERS, TileColor} # doesn't include the guess

# can calculate on-the-fly, but storing might be faster
const feedbacks_all = vec(collect(Iterators.product(fill(instances(TileColor), NLETTERS)...)))
const feedbacks_dict = Dict(fb => i for (i,fb) in enumerate(feedbacks_all))

Base.instances(::Type{Feedback}) = feedbacks_all

const NFEEDBACKS = length(instances(Feedback))
const CORRECT_GUESS = feedbacks_dict[Tuple(fill(GREEN, NLETTERS))]

Base.Int(fb::Feedback) = feedbacks_dict[fb]
Feedback(i::Integer) = feedbacks_all[i]

# allows index with feedbacks directly
Base.to_index(fb::Feedback) = Int(fb)

function Base.show(io::IO, fb::Feedback)
    color_print(io, fill('█', NLETTERS), map(f -> get_color(f), fb))
end

function entropy(fbv::Vector{Feedback})
    ps = zeros(NCOLORS ^ NLETTERS)
    for (k,v) in countmap(fbv)
        ps[k] += v
    end
    ps ./= length(fbv)
    
    return entropy(ps)
end

entropy (generic function with 2 methods)

In [4]:
function get_feedback(guess::String, answer::String)
    guess, answer = collect(guess), collect(answer)
    
    rv = map(1:NLETTERS) do i
        c = guess[i]
        if c == answer[i]
            GREEN
        elseif (c in answer) && (sum(guess[1:(i-1)] .== c) < sum(answer .== c))
            # second condition because of the way WORDLE deals with multiple appearences of a letter
            YELLOW
        else
            GREY
        end
    end
    
    return Tuple(rv)
end

function get_feedback_groups(fbv::Vector{Feedback})
    rv = [Vector{ITYPE}() for _ in 1:length(instances(Feedback))]
    for (i,f) in enumerate(fbv)
        push!(rv[f], i)
    end
    return rv
end

get_feedback_groups (generic function with 1 method)

In [5]:
function make_greedy_guess(feedbacks::AbstractMatrix{Feedback}, wl::AbstractVector{<:Integer})
    # returns word index with the maximum entropy, given the word list
    es = map(wi -> entropy(feedbacks[wi,wl]), 1:NWORDS)
    rv = argmax(es)
    
    # if one of the words in the given list has the maximum entropy, we should return that.
    # for example, if there's only one word in the list, a lot of words could have entropy = 0.
    # choosing one of the words in the list means the terminal node occurs next.
    # if we don't do this, we need to separately check for (length(*) == 1) cases in the `greedy depths` algorithm.
    e = es[rv]
    for wi in wl
        if e ≈ es[wi]
            rv = wi
            break
        end
    end
    
    return ITYPE(rv)
end

function filter_groups(cgroups::Vector{Vector{ITYPE}}, wl::Vector{ITYPE})
    intersect.(cgroups, Ref(wl))
end

function get_greedy_depths(feedbacks, feedback_groups)
    gdepths = zeros(Int, NWORDS)

    # each element: [(guess, depth, fb_groups), fb_state]
    iguess = make_greedy_guess(feedbacks, 1:NWORDS) # initial guess
    S = [[(iguess, 1, feedback_groups[iguess]), 1]]
    while !isempty(S)
        ((guess, depth, fb_groups), fb_state) = S[end]
        if fb_state > NFEEDBACKS # feedbacks exhausted
            pop!(S)
        elseif (fb_state == CORRECT_GUESS) && !isempty(fb_groups[CORRECT_GUESS])
            # correct guess feedback and need to make sure that the word isn't eliminated by earlier guesses
            gdepths[guess] = depth
            S[end][end] += 1
        else
            fb_state = findfirst(i -> (i ≥ fb_state) && !isempty(fb_groups[i]), 1:length(fb_groups))
            if isnothing(fb_state) # no more word groups left!!
                pop!(S)
            else
                S[end][end] = fb_state+1  # updating the state
                grp = fb_groups[fb_state] # get the current word group
                nextguess = make_greedy_guess(feedbacks, grp)
                push!(S, [(nextguess, depth+1, filter_groups(feedback_groups[nextguess], grp)), 1])
            end
        end
    end
    
    return gdepths
end

get_greedy_depths (generic function with 1 method)

In [6]:
words = uppercase.(readlines("./words.txt"))
NWORDS = length(words)

feedbacks = @time [get_feedback(w1, w2) for w1 in words, w2 in words];
feedback_groups = @time [get_feedback_groups(feedbacks[i,:]) for i in 1:NWORDS];

 28.872929 seconds (509.30 M allocations: 29.543 GiB, 10.70% gc time, 1.17% compilation time)
  3.165882 seconds (3.60 M allocations: 970.770 MiB, 2.73% gc time, 3.71% compilation time)


In [7]:
gdepths = @time get_greedy_depths(feedbacks, feedback_groups);

 75.200212 seconds (242.31 M allocations: 103.975 GiB, 13.14% gc time, 1.01% compilation time)


In [8]:
countmap(gdepths)

Dict{Int64, Int64} with 7 entries:
  5 => 660
  4 => 3159
  6 => 65
  7 => 1
  2 => 64
  3 => 1807
  1 => 1